## Install requirements

In [1]:
!pip install -q ddgs==9.5.5 openai==1.107.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 26.6 MB/s eta 0:00:00


## Setup Enviornment variables

In [2]:
import getpass
import os

# os.environ["OPENAI_API_KEY"] = "<OPENAI_API_KEY>"

from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

## Setting Up Tools

In [3]:
from ddgs import DDGS

def search(query):
    """
    perform a search on for the given query and return the results
    :param query: the query to search for
    :return: the search results
    """
    response = DDGS().text(query, max_results=5, region='us-en')
    results = [
        {"snippet": r["body"], "title": r["title"], "link": r["href"]}
        for r in response
    ]

    formatted_results = ""
    for result in results:
        formatted_results += f"Title: {result['title']}\n"
        formatted_results += f"Snippet: {result['snippet']}\n"
        formatted_results += "----\n"

    return formatted_results


In [4]:
def evaluate_expression(operation: str) -> float:
    """
    perform a calculation on the given operation and return the result
    :param operation: the operation to perform, should be compatible to use with eval eg: operation: "1+2"
    :return: the result of the operation
    """
    return eval(operation)

In [5]:
print(search("llama index"))

Title: LlamaIndex - LlamaIndex
Snippet: LlamaIndex is a library and a platform for building agents and workflows that use LLMs and data to perform tasks like question-answering, chatbots, and data extraction. Learn how to use LlamaIndex in Python or TypeScript, explore use cases, and join the community.
----
Title: GitHub - run-llama/llama_index: LlamaIndex is the leading ... What is LlamaIndex - GeeksforGeeks llamaindex (LlamaIndex) - Hugging Face LlamaIndex - Build Knowledge Assistants over your Enterprise Data llama-index · PyPI GitHub - run- llama /llama_index: LlamaIndex is the leading LlamaIndex - LlamaIndex LlamaIndex - LlamaIndex GitHub - run- llama /llama_index: LlamaIndex is the leading GitHub - run- llama /llama_index: LlamaIndex is the leading llamaindex (LlamaIndex) - Hugging Face LlamaIndex · GitHub
Snippet: LlamaIndex (GPT Index ) is a data framework for your LLM application. Building with LlamaIndex typically involves working with LlamaIndex core and a chosen set of int

## Creating Agent

In [6]:
import re
from openai import OpenAI

class Agent:
    def __init__(self, client: OpenAI, tools):
        """
        Initialize the Agent class.

        Args:
        client (OpenAI): The OpenAI client instance.
        tools (list): A list of tool functions.
        """
        self.client = client
        self.tools = tools
        self.memory = []

    def get_agent_prompt(self) -> str:
        """
        Generate the agent prompt string.

        Returns:
        str: The agent prompt string.
        """
        tools_str = "\n".join(
            [f"Action:{tool.__name__}\nAction Input:{tool.__doc__} \n" for tool in self.tools]
        )
        system_prompt = f"""
        You run in a loop of Thought, Action, Observation.
        At the end of the loop you output an Answer
        Use Thought to describe your thoughts about the question you have been asked.
        Use Action to run one of the actions available to you.
        Observation will be the result of running those actions.

        Your available actions are:
          {tools_str}

        Try to break the search query into multiple query for good results

        Always return in this format
        Question: the question
        Thought: your thoughts about the question
        Action: tool_name: the input to the action

        Example session:
        Question: New York is in which country?
        Thought: I should search for new york in web
        Action: search: new york

        You will be called again with this:

        Observation: New york is a city in the United States of America.

        You then output:
        Final Answer: New York is in the United States of America.
        """
        return system_prompt

    def add_agent_steps_in_memory(self, role: str, content: str) -> None:
        """
        Add agent steps to memory.

        Args:
        role (str): The role of the agent step.
        content (str): The content of the agent step.
        """
        self.memory.append({"role": role, "content": content})

    def invoke(self, user_input: str, max_iterations: int = 10) -> str:
        """
        Invoke the agent.

        Args:
        user_input (str): The user input string.
        max_iterations (int): The maximum number of iterations.

        Returns:
        str: The final answer.
        """
        i = 0
        self.add_agent_steps_in_memory("system", self.get_agent_prompt())
        current_message = user_input
        tool_names = [tool.__name__ for tool in self.tools]

        while i < max_iterations:
            print(f"Iteration: {i+1}")
            i += 1
            self.add_agent_steps_in_memory("user", current_message)
            result = self.client.responses.create(
                input=self.memory,
                model="gpt-5-chat-latest",
                temperature=0.2
            )
            agent_output = result.output[0].content[0].text
            print(agent_output)
            self.add_agent_steps_in_memory("assistant", agent_output)

            if "Action" in agent_output:
                action = re.findall(r"Action: ([a-z_]+): (.+)", agent_output, re.IGNORECASE)
                print(action)
                chosen_tool = action[0][0]
                arg = action[0][1]

                if chosen_tool in tool_names:
                    result_tool = eval(f"{chosen_tool}('{arg}')")
                    current_message = f"Observation:\n {result_tool}"
                    print(f"chosen tool: {chosen_tool}")
                    print(f"arg: {arg}")
                    print(current_message)
                else:
                    current_message = "Observation: Tool not found"
                continue

            print("----------------------------------------------------------")

            if "Final Answer" in agent_output:
                return agent_output

In [7]:
client = OpenAI(api_key = os.environ["OPENAI_API_KEY"])
tools = [search, evaluate_expression]
agent = Agent(client, tools)
output = agent.invoke("what is differnce between stock price of microsoft and nvidia in 2025? Which has higher stock price?")
print(f"Final Output-> {output}")

Iteration: 1
Question: what is difference between stock price of microsoft and nvidia in 2025? Which has higher stock price?  
Thought: I need to look up the current stock prices of Microsoft and Nvidia in 2025, then calculate the difference and determine which one is higher.  
Action: search: Microsoft stock price 2025  
Action: search: Nvidia stock price 2025  
[('search', 'Microsoft stock price 2025  '), ('search', 'Nvidia stock price 2025  ')]
chosen tool: search
arg: Microsoft stock price 2025  
Observation:
 Title: Coin Price Forecast MICROSOFT STOCK FORECAST 2025, 2026-2036
Snippet: 2 weeks ago - Microsoft stock price prediction and forecast for near days, 2025 and 2026-2036 years. Short-term and long-term predictions are updated daily
----
Title: Long Forecast MICROSOFT STOCK PRICE PREDICTION 2025, 2026, 2027-2029 - Long Forecast
Snippet: 1 hour ago - Microsoft stock price prediction. Microsoft stock forecast for 2025 , 2026, 2027, 2028 and 2029 Open, maximum and minimum, close